In [1]:
from glob import glob
import librosa
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import random
import os
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation


from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_tp = pd.read_csv('../../data/train_tp.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]


SR = 48000
LEN_10SEC = SR * 10

In [3]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore
set_seed(1234)

In [4]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [5]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 df:pd.DataFrame,
                 data_path:str,
                 melspectrogram_parameters:dict,
                 pcen_parameters:dict
    ):
        self.df =  df
        self.path = data_path
        self.img_size = 256
        self.melspectrogram_parameters = melspectrogram_parameters
        self.pcen_parameters = pcen_parameters
        
        
        self.transform = tr.Compose([
          tr.OneOf([
            tr.GaussianNoiseSNR(min_snr=10),
            tr.PinkNoiseSNR(min_snr=10)
          ]),
          #tr.PitchShift(max_steps=2, sr=SR),
          #tr.TimeStretch(),
          #tr.TimeShift(sr=SR),
          tr.VolumeControl(mode="sine")
        ])
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        species_id = sample['species_id']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        annotated_duration = t_max - t_min
        
        if annotated_duration > 15:
            limit_sec = t_max - 10
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + 10
            
            start = (start_sec - 7) * SR
            end = (end_sec + 8) * SR
        else:
            res_time = 10 - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            back_time = res_time - front_time
            
            start = 0
            end = (t_max + back_time + 10) * SR


                    
        record_path = self.path + recording_id + '.flac'
        y, sr = librosa.load(record_path, sr=None)

        tarnsform_flag = random.choice([True, True, False])
        
        if tarnsform_flag:
            y = self.transform(y[start:end])
        
        
        melspec = librosa.feature.melspectrogram(y, sr=SR, **self.melspectrogram_parameters)
        pcen = librosa.pcen(melspec, sr=SR, **self.pcen_parameters)
        clean_mel = librosa.power_to_db(melspec ** 1.5)
        melspec = librosa.power_to_db(melspec)
        
        norm_melspec = normalize_melspec(melspec)
        norm_pcen = normalize_melspec(pcen)
        norm_clean_mel = normalize_melspec(clean_mel)
        image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)
        
        height, width, _ = image.shape
        limit = width - 937
        
        start = random.randint(0, limit)
        
        image = cv2.resize(image[:, start:start+430, :], (self.img_size*2, self.img_size))
        image = np.moveaxis(image, 2, 0)
        image = (image / 255.0).astype(np.float32)
        
        target = torch.zeros([24], dtype=torch.float32)
        target[species_id] = 1
        
        return image, target

In [6]:
class RfcxTestDataSet(Dataset):
    def __init__(self,
                 df:pd.DataFrame,
                 data_path:str,
                 melspectrogram_parameters:dict,
                 pcen_parameters:dict,
                 val=False
    ):
        self.df =  df
        self.val = val
        self.path = data_path
        self.img_size = 256
        self.melspectrogram_parameters = melspectrogram_parameters
        self.pcen_parameters = pcen_parameters
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx: int):
        sample = self.df.iloc[idx, :]
        recording_id = sample['recording_id']
        record_path = self.path + recording_id + '.flac'
        
        y, sr = librosa.load(record_path, sr=None)
        
        species_id = sample['species_id']
        melspec = librosa.feature.melspectrogram(y, sr=SR, **self.melspectrogram_parameters)
        pcen = librosa.pcen(melspec, sr=SR, **self.pcen_parameters)
        clean_mel = librosa.power_to_db(melspec ** 1.5)

        norm_melspec = normalize_melspec(melspec)
        norm_pcen = normalize_melspec(pcen)
        norm_clean_mel = normalize_melspec(clean_mel)
        image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)
        
        images = []
        
        start = 0
        for i in range(6):
            end = start + 937
            height, width, _ = image.shape
            tmp_image = cv2.resize(image[:, start:end, :], (self.img_size*2, self.img_size))
            tmp_image = np.moveaxis(tmp_image, 2, 0)
            tmp_image = (tmp_image / 255.0).astype(np.float32)
            images.append(tmp_image)
            
            start = end
            
        if self.val == True:
            target = torch.zeros([24], dtype=torch.float32)
            target[species_id] = 1
            return images, target
        else:
            return images

In [7]:
class BaseModel(nn.Module):
    def __init__(self, model_type, model_name, output_size, spectrogram_params, logmel_extractor_params, spec_augmenter_params):
        super().__init__()
        self.model_type = model_type
        
        if model_type == 'res':
            if model_name == 'resnet50':
                base_model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
            else:
                base_model = resnet34(pretrained=True)
            
            layers = list(base_model.children())[:-1]
            self.encoder = nn.Sequential(*layers)

            in_features = base_model.fc.in_features

            self.fc1 = nn.Linear(in_features, in_features, bias=True)
            
            """self.fc1 = nn.Sequential(
                    nn.Linear(in_features, 1024),
                    nn.ReLU(),
                    nn.Dropout(p=0.2),
                    nn.Linear(1024, 1024),
                    nn.ReLU(),
                    nn.Dropout(p=0.2),
                    nn.Linear(1024, num_birds)
                )"""

            self.classification = nn.Linear(in_features, output_size)
            
        elif model_type == 'efficientnet':
            self.features = EfficientNet.from_pretrained(model_name)
            model_code = model_name.split('-')[1]
            
            num_input = {
                'b4': 1792,
                'b2': 1408,
                'b7': 2560
            }
            
            self.features_out = num_input[model_code]
            
            self.classification = nn.Sequential(nn.Linear(num_input[model_code], output_size))
        
    def forward(self, input):
        if self.model_type == 'res':
            x = self.encoder(input)

            x = F.dropout(x, p=0.5, training=self.training)
            x = F.relu_(self.fc1(x))
            
            x = self.classification(x)
            
        elif self.model_type == 'efficientnet':
            x = self.features.extract_features(input)
            x = F.avg_pool2d(x, x.size()[2:]).reshape(-1, self.features_out)
            x = self.classification(x)
            
        return x

In [8]:
def save(fold, model, optim, criterion, file_path="../../model/"):
    if not TEST_NAME in os.listdir(file_path):
        os.mkdir(file_path+TEST_NAME)
    
    
    output_path = file_path + TEST_NAME + '/' + f"{TEST_NAME}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [9]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()

# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

# Sample usage
# y_true = torch.tensor(np.array([[1, 1, 0], [1, 0, 1], [0, 0, 1]]))
# y_score = torch.tensor(np.random.randn(3, 3))
# print(LRAP(y_score, y_true), LWLRAP(y_score, y_true))

In [10]:
TEST_NAME = 'baseline-pic-res'

n_splits = 5
random_state = 1
epochs = 35

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spectrogram_params = {
    'n_fft': 1024,
    'hop_length': 320,
    'win_length': 1024,
    'window': 'hann',
    'center': True,
    'pad_mode': 'reflect',
    'freeze_parameters': True
}

logmel_extractor_params = {
    'sr': SR,
    'n_fft': 1024,
    'n_mels': 64,
    'fmin': 50,
    'fmax': 20000,
    'ref' : 1.0,
    'amin': 1e-10,
    'top_db': None,
    'freeze_parameters': True
}

spec_augmenter_params = {
    'time_drop_width':  64,
    'time_stripes_num': 2,
    'freq_drop_width':  8,
    'freq_stripes_num': 2
}

model_params = {
    'model_type': 'res', # res or efficientnet
    'model_name': 'resnet50',
    'output_size': 24,
    'spectrogram_params': spectrogram_params,
    'logmel_extractor_params': logmel_extractor_params,
    'spec_augmenter_params': spec_augmenter_params,
}

optim_params = {
    'lr': 1e-3,
    'weight_decay': 5e-5,
    'betas': (0.9, 0.999)
}

scheduler_params = {
    'mode': 'max',
    'patience': 1,
    'factor': 0.4,
    'verbose': False
}

data_params = {
    'data_path': '/home/yuigahama/kaggle/rfcx/data/train/',
    'melspectrogram_parameters': {
        'n_mels': 128,
        'fmin': 20,
        'fmax': 16000,
    },
    'pcen_parameters': {
        'gain': 0.98,
        'bias': 2,
        'power': 0.5,
        'time_constant': 0.4,
        'eps': 0.000001,
    }
}

test_data_params = {
    'data_path': '/home/yuigahama/kaggle/rfcx/data/test_tensor_mel_48000/',
    'melspectrogram_parameters': {
        'n_mels': 128,
        'fmin': 20,
        'fmax': 16000,
    },
    'pcen_parameters': {
        'gain': 0.98,
        'bias': 2,
        'power': 0.5,
        'time_constant': 0.4,
        'eps': 0.000001,
    }
}

dataloder_params = {
    'batch_size': 64,
    'num_workers': 15,
    'pin_memory': True,
}

test_dataloder_params = {
    'batch_size': 32,
    'num_workers': 15,
    'pin_memory': True,
    'shuffle':False
}

In [11]:
tta = np.zeros((len(submission), 24))
cv_score = 0

for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp.species_id)):
    print(f'---------- fold {fold_id} ----------')
    
    model = BaseModel(**model_params).to(device)
    optim = Adam(model.parameters(), **optim_params)
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optim, **scheduler_params)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optim, T_max=epochs)
    criterion = nn.BCEWithLogitsLoss()
    
    train_dataset = RfcxDataSet(train_tp.iloc[train_index], **data_params)
    val_dataset = RfcxTestDataSet(train_tp.iloc[val_index], val=True, **data_params)
    test_dataset = RfcxTestDataSet(submission, **test_data_params)

    train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloder_params)
    val_dataloader = DataLoader(val_dataset, shuffle=False, **dataloder_params)
    test_dataloader = DataLoader(test_dataset, **test_dataloder_params)
    
    es = 15
    for epoch in range(1, epochs):
        
        start_time = time.time()
        bast_score = 0
        
        # train
        model.train()
        train_loss = 0
        train_score = 0
        
        for data in train_dataloader:
            image = data[0].to(device)
            label = data[1].to(device)
            
            pred = model(image)
            
            optim.zero_grad()
            loss = criterion(pred, label)
            score = LWLRAP(pred.cpu(), label.cpu())
            
            loss.backward()
            optim.step()
            
            train_loss += loss.item()
            train_score += score
            
        train_loss  /= len(train_dataloader)
        train_score /= len(train_dataloader)
        
        # val
        model.eval()
        val_loss = 0
        val_score = 0
        
        with torch.no_grad():
            for val_data in val_dataloader:
                num = len(val_data[0])
                label = val_data[1]
                tmp = torch.zeros((val_data[0][0].size(0), 24))
                
                for img in val_data[0]:
                    pred = model(img.to(device))
                    tmp += pred.cpu() / num
                
                val_loss += criterion(tmp, label).item()
                val_score += LWLRAP(tmp, label)

                
        val_loss  /= len(val_dataloader)
        val_score /= len(val_dataloader)
        
        duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
        print(f'epoch {epoch:3}| T | loss: {train_loss:.3} | score: {train_score:.3} | V | loss: {val_loss:.3} | score: {val_score:.3} | time: {duration}')

        if bast_score < val_score:
            bast_score = val_score
            bast_path = save(fold_id, model, optim, criterion)
        else:
            es -= 1
            
        scheduler.step()
        
        if es == 0:
            break
        
    cv_score += bast_score / 5
        
    oof = np.zeros((len(test_dataset), 24))
    model = BaseModel(**model_params).to(device)
    
    bast_model_parms = torch.load(bast_path)
    model.load_state_dict(bast_model_parms['model_state_dict'])
    
    with torch.no_grad():
        start = 0
        for i, test_data in enumerate(test_dataloader):
            target_size = test_data[0].size(0)
            tmp_pred = np.zeros((target_size, 24))
            num = len(test_data)
            end = start + target_size
            for img in test_data:
                img = img.to(device)
                pred = model(img)
                
                tmp_pred += torch.sigmoid(pred.cpu().detach()).numpy() / num
                       
            oof[start:end, :] = tmp_pred
            start = end
            
    tta += (oof / n_splits) 

---------- fold 0 ----------


Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


epoch   1| T | loss: 0.235 | score: 0.183 | V | loss: 0.227 | score: 0.204 | time: 0:01:17
epoch   2| T | loss: 0.172 | score: 0.213 | V | loss: 0.172 | score: 0.23 | time: 0:01:19
epoch   3| T | loss: 0.161 | score: 0.28 | V | loss: 0.161 | score: 0.285 | time: 0:01:14
epoch   4| T | loss: 0.154 | score: 0.337 | V | loss: 0.165 | score: 0.283 | time: 0:01:15
epoch   5| T | loss: 0.149 | score: 0.368 | V | loss: 0.161 | score: 0.29 | time: 0:01:16
epoch   6| T | loss: 0.141 | score: 0.416 | V | loss: 0.152 | score: 0.388 | time: 0:01:15
epoch   7| T | loss: 0.136 | score: 0.463 | V | loss: 0.144 | score: 0.414 | time: 0:01:15
epoch   8| T | loss: 0.132 | score: 0.477 | V | loss: 0.14 | score: 0.438 | time: 0:01:17
epoch   9| T | loss: 0.127 | score: 0.514 | V | loss: 0.164 | score: 0.365 | time: 0:01:16
epoch  10| T | loss: 0.125 | score: 0.527 | V | loss: 0.15 | score: 0.379 | time: 0:01:14
epoch  11| T | loss: 0.119 | score: 0.554 | V | loss: 0.149 | score: 0.41 | time: 0:01:15
epoch

Using cache found in /home/yuigahama/.cache/torch/hub/zhanghang1989_ResNeSt_master


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/librosa/core/audio.py", line 146, in load
    with sf.SoundFile(path) as sf_desc:
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/soundfile.py", line 629, in __init__
    self._file = self._open(file, mode_int, closefd)
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/soundfile.py", line 1183, in _open
    _error_check(_snd.sf_error(file_ptr),
  File "/home/yuigahama/anaconda3/lib/python3.8/site-packages/soundfile.py", line 1357, in _error_check
    raise RuntimeError(prefix + _ffi.string(err_str).decode('utf-8', 'replace'))
RuntimeError: Error opening '/home/yuigahama/kaggle/rfcx/data/test_tensor_mel_48000/000316da7.flac': System error.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yuigahama/anaconda3/lib/py

In [ ]:
print(cv_score)

In [ ]:
pred_sub = pd.DataFrame(tta, columns=pred_target)
pred_sub

In [ ]:
pd.concat([submission['recording_id'], pred_sub], axis=1).to_csv('submission.csv', index=False)